# Data Implementation For Quasars

## Part I - To find QSO in database and create the txt file (not necessary if created)

In [ ]:
#Query the SDSS database to find QSO
query_result = SDSS.query_sql("""
    SELECT
        plate, mjd, fiberid, z
    FROM
        specObj
    WHERE
        class = 'QSO'
""")

# Print the first few rows of the query result
print(query_result[:5])
#Print length of result found
print("\nNb quasars in SDSS database :",len(query_result))

In [ ]:
#WARNING : TO LAUNCH ONLY IF THE "qso.txt" FILE IS TO BE UPDATED
#VERY LONG
directory = '/arc/projects/k-pop/spectra/boss/lite'
qso_files = []
redshift = []
wavelength = []
flux = []
ivar = []
mask = []

# Indexer files by a reducted format name
file_index = {}
for dossier_parent, _, fichiers in os.walk(directory):
    for nom_fichier in fichiers:
        if nom_fichier.startswith("spec-") and nom_fichier.endswith(".fits"):
            # Extraire le numéro de plaque, de MJD et de fibre du nom de fichier
            parts = nom_fichier.split('-')
            plate = parts[1]
            mjd = parts[2]
            fiberid = parts[3].split('.')[0]  # Retirer l'extension .fits
            # Forcer fiberid à être représenté par quatre chiffres
            fiberid = fiberid.zfill(4)
            file_index[f"spec-{plate}-{mjd}-{fiberid}.fits"] = os.path.join(dossier_parent, nom_fichier)

# Processus for each QSO
for qso in tqdm(qso_list, desc="Processing items"):
    plate, mjd, fiberid = qso['plate'], qso['mjd'], qso['fiberid']
    # Force fiberid to have for digits
    nom_fichier = f"spec-{plate}-{mjd}-{fiberid:04d}.fits"
    if nom_fichier in file_index:
        filepath = file_index[nom_fichier]
        qso_files.append(filepath)
        redshift.append(qso['z'])
        # Lire le fichier FITS
        with fits.open(filepath) as hdul:
            data = hdul[1].data

        # Extract necessary columns
        wavelength.append(np.power([10]*len(data['loglam']), data['loglam']))
        flux.append(data['FLUX'])
        ivar.append(data['IVAR'])
        mask.append(data['AND_MASK'])

print("Nb quasars in our database :", len(qso_files))

In [ ]:
#to place qso_files in qso.txt
# path to the txt file
chemin_fichier_texte = "/arc/home/amolinard/qso.txt"

with open(chemin_fichier_texte, 'w') as f:
    for qso in qso_files:
        f.write(qso + "\n")

## Part 2 - Creation of the HDF5 file

In [7]:
import h5py
import os as os
from tqdm import tqdm 
from astropy.io import fits
import numpy as np

max_length = 4621

#From fits to hdf5
def read_txt_file(path_file):
    with open(path_file, 'r') as f:
        path = f.read().splitlines()
    return path

def ensure_native_byteorder(array):
    if array.dtype.byteorder not in ('=', '|'):  # '=' means native, '|' means not applicable
        return array.byteswap().newbyteorder()  # Swap byte order to native
    return array

def create_mask(flux, ivar):
    """
    Creates a mask for the flux array where the mask is 0 if the flux is zero or sigma > 0.5, and 1 otherwise.
    """
    mask = np.where((flux == 0) | (1/ivar > 0.5), 0, 1)
    return mask

def convert_fits_to_hdf5(hdf5_path, max_files=300, save_interval=10):
    count = 0
    with h5py.File(hdf5_path, 'w') as hdf5_file:
        all_files = read_txt_file('/arc/home/amolinard/qso.txt')
        all_files = all_files[:max_files]  # Limit the number of files
        for i in tqdm(range(0, len(all_files), save_interval), desc="Converting FITS to HDF5"):
            for file_name in all_files[i:i+save_interval]:
                file_path = file_name
                with fits.open(file_path) as hdul:
                    spectrum_index = count
                    count = count + 1
                    data = hdul[1].data
                    add_data = hdul['SPALL'].data
                    redshift = add_data['Z'][0]
                    flux = data['FLUX']
                    ivar = data['IVAR']
                    length = len(flux)
                    wavelength = np.power([10]*len(data['loglam']),data['loglam'])

                    
                    #To have data of the same dimension
                    padded_flux = np.pad(flux, (0, max_length-len(flux)), mode='constant', constant_values=0)
                    padded_ivar = np.pad(ivar, (0, max_length-len(ivar)), mode='constant', constant_values=0)
                    padded_wavelength = np.pad(wavelength, (0, max_length-len(wavelength)), mode='constant', constant_values=0)

                    flux = ensure_native_byteorder(padded_flux)
                    ivar = ensure_native_byteorder(padded_ivar)
                    wavelength = ensure_native_byteorder(padded_wavelength)
                    
                    flux_mask = create_mask(padded_flux, padded_ivar).astype(np.float32)

                    short_file_name = os.path.basename(file_name)
                    grp = hdf5_file.create_group(short_file_name)
                    grp.create_dataset('flux', data=padded_flux)
                    grp.create_dataset('wavelength', data=padded_wavelength)
                    grp.create_dataset('mask', data=flux_mask)
                    grp.create_dataset('ivar', data=padded_ivar)
                    grp.create_dataset('redshift', data=redshift)
                    grp.create_dataset('length', data=length)
                    grp.create_dataset('spectrum_index', data=spectrum_index)
            
            hdf5_file.flush()  # Ensure data is written to disk

if __name__ == "__main__":
    convert_fits_to_hdf5('/arc/home/amolinard/Padded_Database.hdf5',60000)

Converting FITS to HDF5:   0%|          | 0/5371 [00:00<?, ?it/s]/tmp/ipykernel_1181/2380434420.py:24: RuntimeWarning: divide by zero encountered in divide
  mask = np.where((flux == 0) | (1/ivar > 0.5), 0, 1)
Converting FITS to HDF5: 100%|██████████| 5371/5371 [1:43:54<00:00,  1.16s/it]  


In [19]:
import h5py
import os as os
from tqdm import tqdm 
from astropy.io import fits
import numpy as np

max_length = 4621

#From fits to hdf5
def read_txt_file(path_file):
    with open(path_file, 'r') as f:
        path = f.read().splitlines()
    return path

def ensure_native_byteorder(array):
    if array.dtype.byteorder not in ('=', '|'):  # '=' means native, '|' means not applicable
        return array.byteswap().newbyteorder()  # Swap byte order to native
    return array

def create_mask(flux, ivar):
    """
    Creates a mask for the flux array where the mask is 0 if the flux is zero or sigma > 0.5, and 1 otherwise.
    """
    mask = np.where((flux == 0) | (1/ivar > 0.5), 0, 1)
    return mask

def convert_fits_to_hdf5(hdf5_path, max_files=300, save_interval=10):
    count = 0
    with h5py.File(hdf5_path, 'w') as hdf5_file:
        all_files = read_txt_file('/arc/home/amolinard/qso.txt')
        all_files = all_files[:max_files]  # Limit the number of files
        file_names = hdf5_file.create_dataset('file_names', (max_files,), dtype=h5py.string_dtype())
        for i in tqdm(range(0, len(all_files), save_interval), desc="Converting FITS to HDF5"):
            for file_name in all_files[i:i+save_interval]:
                file_path = file_name
                with fits.open(file_path) as hdul:
                    spectrum_index = count
                    count = count + 1
                    data = hdul[1].data
                    add_data = hdul['SPALL'].data
                    redshift = add_data['Z'][0]
                    flux = data['FLUX']
                    ivar = data['IVAR']
                    length = len(flux)
                    wavelength = np.power(10, data['loglam'])
                    
                    #To have data of the same dimension
                    padded_flux = np.pad(flux, (0, max_length-len(flux)), mode='constant', constant_values=0)
                    padded_ivar = np.pad(ivar, (0, max_length-len(ivar)), mode='constant', constant_values=0)
                    padded_wavelength = np.pad(wavelength, (0, max_length-len(wavelength)), mode='constant', constant_values=0)

                    flux = ensure_native_byteorder(padded_flux)
                    ivar = ensure_native_byteorder(padded_ivar)
                    wavelength = ensure_native_byteorder(padded_wavelength)
                    
                    flux_mask = create_mask(padded_flux, padded_ivar).astype(np.float32)

                    short_file_name = os.path.basename(file_name)
                    file_names[spectrum_index] = short_file_name
                    grp = hdf5_file.create_group(short_file_name)
                    grp.create_dataset('flux', data=padded_flux)
                    grp.create_dataset('wavelength', data=padded_wavelength)
                    grp.create_dataset('mask', data=flux_mask)
                    grp.create_dataset('ivar', data=padded_ivar)
                    grp.create_dataset('redshift', data=redshift)
                    grp.create_dataset('length', data=length)
                    grp.create_dataset('spectrum_index', data=spectrum_index)
            
            hdf5_file.flush()  # Ensure data is written to disk

if __name__ == "__main__":
    convert_fits_to_hdf5('/arc/home/amolinard/Padded_Database.hdf5',60000)

Converting FITS to HDF5:   0%|          | 0/5371 [00:00<?, ?it/s]/tmp/ipykernel_1181/4164100400.py:24: RuntimeWarning: divide by zero encountered in divide
  mask = np.where((flux == 0) | (1/ivar > 0.5), 0, 1)
Converting FITS to HDF5: 100%|██████████| 5371/5371 [2:40:56<00:00,  1.80s/it]  
